# py0xcluster

This is for now an educational project for myself with the aim of performing fun data-science projects around blockchain data gathered through the Graph Network (https://thegraph.com/)

## Target objectives

- Establish meaningful grouping of address by clustering DEX traders and LP
    - Feature Extraction:
        - TBD but based on mint/swap/burn data from messari subgraphs entities
        - with or without balances at swap time (web3py fetch balance at block)
        - EOA vs Contracts
    - Dimensionality reduction:
        - UMAP / tSNE or PCA / ICA
    - Clustering:
        - DBSCAN
        - silhouette evaluation
    - Visualization:
        - scatter plot with color-coded returns? (TBD)

**Secondary objective: identify which group has the most profitable activity**

- Triggered Average of price by swap in/out by group of addresses

- Predict future returns based on the activity of previously clustered groups of addresses


### Imports

In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from py0xcluster.utils.query_utils import *
from py0xcluster.main_classes.pools import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Gathering data about most-active pools

In [8]:
uni3pools_selector = PoolSelector(
    subgraph_url = 'https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum',
    min_daily_volume_USD = 100000,
    min_TVL = 100000, # Not implemented. consider removing
    start_date = (2022,12,21), 
    end_date = (2023,1,10),
    days_batch_size = 20)

uni3_pools = uni3pools_selector.create_pool_selection()
uni3_pools.pools_df

Queriying from 2022-12-21 00:00:00 to 2023-01-10 00:00:00
1650 lquidity pools snapshots retrieved
197 stable pools snapshots (over 1650) have been removed
217 illiquid pools snapshots (over 1650) have been removed 
221 pools were selected


,pool.name,dailyVolumeUSD,pool.totalValueLockedUSD,token0.lastPriceUSD,token1.lastPriceUSD,pool.protocol.name,pool.protocol.network,pool.id,token0.symbol,token1.symbol
0,Uniswap V3 USD Coin/Wrapped Ether 0.05%,1.407573e+08,1.790617e+08,1.000000,1334.218011,Uniswap V3,MAINNET,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,WETH
1,Uniswap V3 USD Coin/Tether USD 0.01%,7.632963e+07,1.200383e+08,1.000000,1.000000,Uniswap V3,MAINNET,0x3416cf6c708da44db2624d63ea0aaef7113527c6,USDC,USDT
2,Uniswap V3 Wrapped Ether/Tether USD 0.05%,2.748191e+07,1.992092e+07,1334.218011,1.000000,Uniswap V3,MAINNET,0x11b815efb8f581194ae79006d24e0d814b7697f6,WETH,USDT
3,Uniswap V3 USD Coin/Wrapped Ether 0.01%,1.951618e+07,5.123749e+06,1.000000,1334.218011,Uniswap V3,MAINNET,0xe0554a476a092703abdb3ef35c80e0d76d32939f,USDC,WETH
4,Uniswap V3 Wrapped BTC/Wrapped Ether 0.05%,1.490067e+07,1.114253e+08,17414.239686,1334.218011,Uniswap V3,MAINNET,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,WBTC,WETH
...,...,...,...,...,...,...,...,...,...,...
216,Uniswap V3 Green/Wrapped Ether 0.05%,1.030147e+05,6.053919e+05,0.002071,1334.218011,Uniswap V3,MAINNET,0x4efc9e2e3e77732ce2f9612b8f050082c01688bd,GREEN,WETH
217,Uniswap V3 HuntToken/Tether USD 0.3%,1.028683e+05,9.968453e+05,0.248318,1.000000,Uniswap V3,MAINNET,0x54578b6f942aeb23b67a8cef24220651306b8e26,HUNT,USDT
218,Uniswap V3 Index/Wrapped Ether 1%,1.026767e+05,2.797922e+04,0.000000,1334.218011,Uniswap V3,MAINNET,0x8c13148228765ba9e84eaf940b0416a5e349a5e7,INDEX,WETH
219,Uniswap V3 unification.com/xfund/Wrapped Ether 1%,1.023294e+05,2.742566e+04,0.000000,1334.218011,Uniswap V3,MAINNET,0xb1223da8a5929bcfa9d26f0c6da8f0a29c3925ff,xFUND,WETH


### Filtering pools
    - more exactly filtering snapshots before processing

In [88]:
min_TVL = 500000

uni3_pools = uni3pools_selector.remove_illiquid_pools(uni3_pools, min_TVL)
uni3_pools = uni3pools_selector.remove_stable_pools(uni3_pools)

# After this point uni3_pools dataframe changes structure to become an aggregation
# (only run once or query again raw data)
uni3_pools = uni3pools_selector.aggregate_snapshots(uni3_pools)

uni3_pools

2839 illiquid pools snapshots (over 7526) have been removed 
730 stable pools snapshots (over 4687) have been removed


,pool.name,dailyVolumeUSD,pool.totalValueLockedUSD,token0.lastPriceUSD,token1.lastPriceUSD,pool.protocol.name,pool.protocol.network,pool.id,token0.symbol,token1.symbol
0,Uniswap V3 USD Coin/Wrapped Ether 0.05%,2.626963e+08,1.787315e+08,1.000000,1337.925269,Uniswap V3,MAINNET,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,WETH
1,Uniswap V3 Wrapped BTC/Wrapped Ether 0.05%,3.683370e+07,1.114904e+08,17418.882366,1337.925269,Uniswap V3,MAINNET,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,WBTC,WETH
2,Uniswap V3 Wrapped Ether/Tether USD 0.05%,3.380961e+07,3.468095e+07,1337.925269,1.000000,Uniswap V3,MAINNET,0x11b815efb8f581194ae79006d24e0d814b7697f6,WETH,USDT
3,Uniswap V3 USD Coin/Wrapped Ether 0.3%,1.788214e+07,1.355260e+08,1.000000,1337.925269,Uniswap V3,MAINNET,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,USDC,WETH
4,Uniswap V3 USD Coin/Wrapped Ether 0.01%,1.082587e+07,5.052765e+06,1.000000,1337.925269,Uniswap V3,MAINNET,0xe0554a476a092703abdb3ef35c80e0d76d32939f,USDC,WETH
...,...,...,...,...,...,...,...,...,...,...
104,Uniswap V3 Tether USD/AVINOC Token 0.3%,1.180537e+05,5.663719e+06,1.000000,0.228010,Uniswap V3,MAINNET,0x2eb8f5708f238b0a2588f044ade8dea7221639ab,USDT,AVINOC
105,Uniswap V3 Status Network Token/Wrapped Ether ...,1.155646e+05,1.470164e+06,0.021328,1337.925269,Uniswap V3,MAINNET,0x084b5191bd08412952337b1108b6e5942418928f,SNT,WETH
106,Uniswap V3 USD Coin/AVINOC Token 0.3%,1.131157e+05,6.873842e+06,1.000000,0.228010,Uniswap V3,MAINNET,0x04a2004b2032fef2ba93f40b0e34d26ab7b00120,USDC,AVINOC
107,Uniswap V3 Green/Wrapped Ether 0.05%,1.081729e+05,6.052408e+05,0.002039,1337.925269,Uniswap V3,MAINNET,0x4efc9e2e3e77732ce2f9612b8f050082c01688bd,GREEN,WETH
